In [251]:
import torch
import json

In [252]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/MaxMinData.json', 'r') as file:
    data = json.load(file)
PDmax = data['PDmax']
PDmin = data['PDmin']
Tmax = data['Tmax']
Tmin = data['Tmin']
ACCmax = data['ACCmax']
ACCmin = data['ACCmin']

In [253]:
PDmax, PDmin, Tmax, Tmin, ACCmax, ACCmin

(1.9444999999999708, 0.0, 30.5, 25.0, 32.0, 21.0)

In [254]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def ACC_normalize(ACC):
    return ((ACC - ACCmin) / (ACCmax - ACCmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

def ACC_denormalize(ACC):
    return ((ACCmax - ACCmin) * ACC + ACCmin)

In [255]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

LSTM(
  (lstm): LSTM(10, 32, batch_first=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [256]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')
ori_data.shape

torch.Size([511, 30, 10])

In [257]:
ori_data[100][0][0]

tensor(0.4400)

In [258]:
temp_denormalize(ori_data[100][0][0])

tensor(27.4200)

In [259]:
for i in range(0, 30, 10):
    print('室內溫度 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_denormalize(j)
        ori_data[i][-3][2] = jj
        ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)
        

室內溫度 : 30.0875
21 度: 32.2814
22 度: 32.289
23 度: 32.2976
24 度: 32.3072
25 度: 32.3176
26 度: 32.3286
27 度: 32.3402
32 度: 32.4005
室內溫度 : 29.4
21 度: 32.2134
22 度: 32.2255
23 度: 32.2383
24 度: 32.2515
25 度: 32.265
26 度: 32.2787
27 度: 32.2925
32 度: 32.3599
室內溫度 : 28.8775
21 度: 32.1599
22 度: 32.1751
23 度: 32.1905
24 度: 32.2059
25 度: 32.2214
26 度: 32.2368
27 度: 32.252
32 度: 32.324
